FAISS Set Up

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

/Users/azamkhan/anaconda3/envs/museum/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

Elara wasn't your typical twelve-year-old. While other kids dreamed of ponies and pop stars, Elara dreamed of unexplored jungles and forgotten cities. Her dreams, however, were consistently hampered by her perpetually cluttered, second-hand backpack, more hole than fabric. That all changed the day she found it: a backpack woven from midnight-blue starlight, shimmering with an almost imperceptible glow.

It was tucked away in the dusty attic of her grandmother's rambling Victorian house, nestled amongst moth-eaten shawls and chipped porcelain dolls.  The moment Elara touched it, a warmth, like a thousand tiny suns, spread through her.  When she opened it, it wasn't empty; it was filled with… possibilities.

At first, the possibilities were small. A perfectly ripe mango appeared, then a sketchbook filled with charcoal sketches she hadn't drawn, then a book detailing the lost language of the Amazonian Yagua people. The backpack seemed to anticipate her needs, providing exactly what she la

In [1]:
!pip install numpy
!pip install sentence-transformers
!pip install openai
!pip install faiss-cpu
!pip install transformers

  Using cached numpy-2.1.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.1.3-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.5.1-cp311-none-macosx_11_0_arm64.whl.metadata (28 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2

In [7]:
import json  # Built-in, no installation needed
import numpy as np
from sentence_transformers import SentenceTransformer
import openai
import faiss  # For building and querying the vector database
from transformers import pipeline
import os
import openai
import getpass
import faiss
import numpy as np

In [8]:
from sentence_transformers import SentenceTransformer
import json

model = SentenceTransformer('all-MiniLM-L6-v2')  # A



In [9]:


# Load the JSON file
with open("metdata.json", "r") as file:
    data = json.load(file)

# Extract paintings data
paintings = data["paintings"]

# Initialize the embedding model
# lightweight model for embeddings

# Generate embeddings for each painting
texts = [
    f"{painting['title']} {painting['description']} {' '.join(painting.get('tags', []))}"
    for painting in paintings
]

# Encode the texts to create embeddings
embeddings = model.encode(texts, convert_to_tensor=False)  # Returns a NumPy array

# Display the first embedding for verification
print(f"First embedding for '{paintings[0]['title']}':", embeddings[0])

First embedding for 'Joan of Arc': [ 1.28659364e-02  7.52580985e-02  2.63640340e-02 -6.11152127e-02
  5.59599064e-02  1.04252696e-01 -4.39553671e-02  6.11743815e-02
  2.49664001e-02 -5.65736070e-02  2.34689694e-02 -6.01532832e-02
  5.46257161e-02 -3.85121405e-02 -2.49281917e-02 -8.73399060e-03
 -4.41297963e-02  1.16389252e-01 -4.19332534e-02  8.00186992e-02
 -5.08325696e-02 -6.55834526e-02  2.17069872e-02  2.89588682e-02
  8.65611732e-02 -3.24539207e-02  3.39158848e-02 -4.38671298e-02
 -5.75492755e-02  6.13340512e-02 -4.35424550e-03  1.16772130e-02
 -8.80586281e-02  2.52330173e-02 -6.72517419e-02 -6.30604476e-02
  1.71815255e-03  2.78348587e-02 -4.09635603e-02  6.47060722e-02
 -1.01237446e-01 -4.32757102e-02 -8.72333944e-02 -7.28660673e-02
  5.87236099e-02 -4.21024226e-02  2.00012643e-02  3.00585534e-02
  1.56338383e-02  1.17664738e-02  7.85477981e-02  4.31159474e-02
 -2.50382232e-03 -4.93614934e-02 -7.88974985e-02 -3.49021852e-02
  4.77681197e-02  1.61563754e-02  4.05374728e-02  6.142

In [10]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
embedding_array = np.array(embeddings).astype('float32')

# Initialize FAISS index
dimension = embedding_array.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)
index.add(embedding_array)  # Add embeddings to the index

In [7]:
faiss.write_index(index, "paintings_index.faiss")


In [25]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your GPT API key: ")

In [26]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [11]:
# Load the FAISS index
index = faiss.read_index("paintings_index.faiss")

# Load the JSON metadata
with open("metdata.json", "r") as file:
    data = json.load(file)

paintings = data["paintings"]  # List of painting metadata

In [12]:
# Search function
def search_paintings(query, top_k=3):
    """
    Searches the FAISS index for the most relevant paintings based on a user query.

    Args:
        query (str): The user's input query.
        top_k (int): Number of top results to return.
    
    Returns:
        list: List of dictionaries containing painting metadata.
    """
    query_embedding = model.encode([query], convert_to_tensor=False)
    query_embedding = np.array(query_embedding).astype('float32')
    distances, indices = index.search(query_embedding, top_k)  # FAISS search
    results = [paintings[i] for i in indices[0]]  # Retrieve painting metadata
    return results

In [14]:
search_paintings("At Table", top_k=3)

[{'id': 'empty',
  'title': '',
  'artist': '',
  'year': '',
  'description': '',
  'location': {'room': '', 'description': ''},
  'tags': [],
  'visual_description': {'dominant_colors': [],
   'size': '',
   'style': '',
   'notable_elements': ''},
  'narrative': {'general': '',
   'historical_context': '',
   'symbolic_meaning': '',
   'conversational_prompts': ['']},
  'expert_knowledge': [{'expert_name': '', 'credentials': '', 'insight': ''},
   {'expert_name': '', 'credentials': '', 'insight': ''}],
  'things_to_mention': '',
  'web_links': [''],
  'related_paintings': [{'painting': '',
    'artist': '',
    'year': '',
    'reason': '',
    'location_from_painting': ''}]},
 {'id': '16',
  'title': 'At Table',
  'artist': 'Edouard Vuillard',
  'year': '1893',
  'description': "Vuillard is best known for his small, poetic, and dimly lit intimiste interiors of the 1890s. The setting of these works were the modest Parisian apartments that the artist shared with his sisters and his m

In [ ]:
client = openai.OpenAI(
    api_key = os.getenv("OPENAI_API_KEY")
)

In [ ]:
client = openai.OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [12]:
# Generate response function
def generate_response(query):
    """
    Generates a conversational response based on the user's query and related paintings.

    Args:
        query (str): The user's input query.
    
    Returns:
        str: The generated conversational response.
    """
    # Search for relevant paintings
    results = search_paintings(query)

    # Construct a conversational prompt
    painting_info = "\n".join(
        [f"Title: {p['title']}\nDescription: {p['description']}" for p in results]
    )
    print(painting_info)
    prompt = f"""
    User asked: {query}
    Here is some information about related paintings:
    {painting_info}
    Please respond with the paintings that are listed above. 
    Respond to the user in a conversational tone and provide a brief history of the painting. Then ask the user if they have any questions about this painting.
    """
    
    # Generate response from the OpenAI API
    response = client.chat.completions.create(
    model="gemini-1.5-flash",
    n=1,
    # messages=[
    #         {"role": "system", "content": prompt}
    #     ]
    messages=[
        {"role": "system", "content": "You are a helpful museum guide."},
        {
            "role": "user",
            "content": prompt
        }
    ]
    )
    return response.choices[0].message.content

# Test the response generation
query = "What painting is the one with three girls in red dresses peeling potatoes?"
response = generate_response(query) 
print(response)

Title: The Three Sisters
Description: In the 1890s Frederic’s paintings of impoverished workers and peasants in his native Belgium were celebrated for their forthrightness and arresting intensity. Here, the humdrum activity of peeling potatoes is vivified by the girls’ bright red dresses and gleaming red-gold and blond hair. Their downcast eyes and serene expressions recall representations of the young Virgin Mary in sixteenth-century Flemish art, which Frederic greatly admired. Nothing is known of the sitters beyond the painting’s title, which identifies them as sisters; the two eldest are so uncannily alike that they appear to be twins.
Title: La Coiffure
Description: This painting shows a woman arranging another woman's hair and a child in the foreground, with simplified forms and earthy colors that convey a sense of timelessness. Picasso’s focus on the figure’s solid, sculptural quality reflects his interest in the classical and primitive influences during this period.
Title: The G

In [11]:
#do another test
query = "What painting should I look at after The Three Sisters? and how do i find it in the museum?"
response = generate_response(query) 
print(response)

Title: The Three Sisters
Description: In the 1890s Frederic’s paintings of impoverished workers and peasants in his native Belgium were celebrated for their forthrightness and arresting intensity. Here, the humdrum activity of peeling potatoes is vivified by the girls’ bright red dresses and gleaming red-gold and blond hair. Their downcast eyes and serene expressions recall representations of the young Virgin Mary in sixteenth-century Flemish art, which Frederic greatly admired. Nothing is known of the sitters beyond the painting’s title, which identifies them as sisters; the two eldest are so uncannily alike that they appear to be twins.
Title: Young Mother Gazing at Her Child
Description: This doting mother in picturesque dress and her angelic offspring are two of many such figures that Bouguereau painted with an eye to the international market. The scene is highly similar in subject and composition to Breton Brother and Sister, also dated 1871 (hanging nearby), and both works were b

In [36]:
# write an api request to get the date from this api: https://collectionapi.metmuseum.org/public/collection/v1/objects/[objectID]

import requests
import json
from pprint import pprint

def get_date(objectID):
    url = f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{objectID}"
    response = requests.get(url)
    data = response.json()
    return data

obj = (get_date(436076)) # 1930
pprint(obj)

{'GalleryNumber': '827',
 'accessionNumber': '31.132.34',
 'accessionYear': '1931',
 'additionalImages': [],
 'artistAlphaSort': 'Dagnan-Bouveret, Pascal-Adolphe-Jean',
 'artistBeginDate': '1852',
 'artistDisplayBio': 'French, Paris 1852–1929 Quincey',
 'artistDisplayName': 'Pascal-Adolphe-Jean Dagnan-Bouveret',
 'artistEndDate': '1929',
 'artistGender': '',
 'artistNationality': 'French',
 'artistPrefix': '',
 'artistRole': 'Artist',
 'artistSuffix': '',
 'artistULAN_URL': 'http://vocab.getty.edu/page/ulan/500022873',
 'artistWikidata_URL': 'https://www.wikidata.org/wiki/Q663837',
 'city': '',
 'classification': 'Paintings',
 'constituents': [{'constituentID': 107550,
                   'constituentULAN_URL': 'http://vocab.getty.edu/page/ulan/500022873',
                   'constituentWikidata_URL': 'https://www.wikidata.org/wiki/Q663837',
                   'gender': '',
                   'name': 'Pascal-Adolphe-Jean Dagnan-Bouveret',
                   'role': 'Artist'}],
 'country